In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# 1. Load dataset
# -----------------------
file_path = r"C:\Users\ishan\Desktop\Master projects\Digital SCVC\online_retail.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")


In [3]:
# 2. Data cleaning
# -----------------------
df = df.dropna(subset=["InvoiceNo", "StockCode", "Quantity", "InvoiceDate", "UnitPrice"])
df = df[df["Quantity"] > 0]  # remove cancellations
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["Sales"] = df["Quantity"] * df["UnitPrice"]


In [4]:
# 3. Synthetic inventory
# -----------------------
df = df.sort_values(["StockCode", "InvoiceDate"])
df["InitialInventory"] = 1000
df["CumulativeSold"] = df.groupby("StockCode")["Quantity"].cumsum()
df["InventoryOnHand"] = df["InitialInventory"] - df["CumulativeSold"]


In [5]:
# 4. Simulate deliveries
# -----------------------
np.random.seed(42)
df["PlannedLeadTime"] = 5
df["ActualLeadTime"] = np.random.randint(3, 9, size=len(df))
df["DeliveryStatus"] = np.where(df["ActualLeadTime"] <= df["PlannedLeadTime"], "On Time", "Delayed")

In [6]:
# 5. Monthly sales aggregation
# -----------------------
monthly_sales = df.set_index("InvoiceDate").resample("M")["Sales"].sum().reset_index()
monthly_sales["MonthIndex"] = np.arange(len(monthly_sales))

C:\Users\ishan\AppData\Local\Temp\ipykernel_121296\1090288301.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.set_index("InvoiceDate").resample("M")["Sales"].sum().reset_index()


In [7]:
# Forecast using Linear Regression
X = monthly_sales[["MonthIndex"]]
y = monthly_sales["Sales"]
model = LinearRegression()
model.fit(X, y)
monthly_sales["ForecastSales"] = model.predict(X)


In [8]:
# 6. Executive KPIs
# -----------------------
avg_delivery_time = df["ActualLeadTime"].mean()
total_inventory_value = (df["InventoryOnHand"] * df["UnitPrice"]).sum()
on_time_percentage = (df["DeliveryStatus"]=="On Time").mean()*100

print("Avg delivery time:", round(avg_delivery_time,2))
print("Total inventory value:", round(total_inventory_value,2))
print("% of orders delivered on time:", round(on_time_percentage,2))

Avg delivery time: 5.5
Total inventory value: -1691284745.65
% of orders delivered on time: 49.97


In [9]:
# 7. Save processed data for Streamlit
# -----------------------
df.to_csv(r"C:\Users\ishan\Desktop\Master projects\Digital SCVC\processed_data.csv", index=False)
monthly_sales.to_csv(r"C:\Users\ishan\Desktop\Master projects\Digital SCVC\monthly_sales.csv", index=False)